# Graph-Conditioned LoRA Evaluation

This notebook evaluates the trained graph-conditioned LoRA model by:
1. Loading the checkpoint
2. Generating images with different scene graphs
3. Comparing results visually
4. Analyzing if the model learned to respect graph structure

In [ ]:
import torch
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

# Find the latest checkpoint
checkpoint_dir = Path("/home/namanb/SBILab/CSE677/Project/SGRel-DiT-Scene-Graph-Aware-Relation-Analysis-in-Diffusion-Transformers/relation-analysis/scripts/checkpoints/graph_lora_diffusion/final_graph_lora_token.pt")
# checkpoints = sorted(checkpoint_dir.glob("checkpoint_epoch_*.pt"))/

if checkpoints:
    latest_checkpoint = checkpoints[-1]
    print(f"Found checkpoint: {latest_checkpoint}")
    print(f"Checkpoint epoch: {latest_checkpoint.stem.split('_')[-1]}")
else:
    print("No checkpoints found!")

No checkpoints found!


## Generate Test Images

Run the inference script to generate images with different scene graphs.

In [7]:
# Run inference to generate test images
import subprocess

cmd = f"""
cd scripts && python test_graph_lora_inference.py \\
    --checkpoint /home/namanb/SBILab/CSE677/Project/SGRel-DiT-Scene-Graph-Aware-Relation-Analysis-in-Diffusion-Transformers/relation-analysis/scripts/checkpoints/graph_lora_diffusion/final_graph_lora_token.pt/home/namanb/SBILab/CSE677/Project/SGRel-DiT-Scene-Graph-Aware-Relation-Analysis-in-Diffusion-Transformers/relation-analysis/scripts/checkpoints/graph_lora_diffusion/final_graph_lora_token.pt/home/namanb/SBILab/CSE677/Project/SGRel-DiT-Scene-Graph-Aware-Relation-Analysis-in-Diffusion-Transformers/relation-analysis/scripts/checkpoints/graph_lora_diffusion/final_graph_lora_token.pt \\
    --device cuda:3 \\
    --height 512 \\
    --width 512 \\
    --output-dir ../inference_results
"""

print("Running inference...")
print("This will generate images with different scene graphs")
print("Command:", cmd)
print("\nUncomment below to run:")
print("# !{cmd}")

Running inference...
This will generate images with different scene graphs
Command: 
cd scripts && python test_graph_lora_inference.py \
    --checkpoint /home/namanb/SBILab/CSE677/Project/SGRel-DiT-Scene-Graph-Aware-Relation-Analysis-in-Diffusion-Transformers/relation-analysis/scripts/checkpoints/graph_lora_diffusion/final_graph_lora_token.pt/home/namanb/SBILab/CSE677/Project/SGRel-DiT-Scene-Graph-Aware-Relation-Analysis-in-Diffusion-Transformers/relation-analysis/scripts/checkpoints/graph_lora_diffusion/final_graph_lora_token.pt/home/namanb/SBILab/CSE677/Project/SGRel-DiT-Scene-Graph-Aware-Relation-Analysis-in-Diffusion-Transformers/relation-analysis/scripts/checkpoints/graph_lora_diffusion/final_graph_lora_token.pt \
    --device cuda:3 \
    --height 512 \
    --width 512 \
    --output-dir ../inference_results


Uncomment below to run:
# !{cmd}


## Compare Generated Images

Visual comparison of images with different scene graphs.

In [ ]:
results_dir = Path("./inference_results")

# Define comparison groups (same entities, different relationships)
comparison_groups = [
    {
        "title": "Person & Bike: Different Relationships",
        "images": [
            ("person_riding_bike.png", "riding"),
            ("person_on_bike.png", "on"),
            ("person_next_to_bike.png", "next to")
        ]
    },
    {
        "title": "Dog & Hat: Wearing vs Next To",
        "images": [
            ("dog_wearing_hat.png", "wearing"),
            ("dog_next_to_hat.png", "next to")
        ]
    },
    {
        "title": "Person & Cup: Different Actions",
        "images": [
            ("person_holding_cup.png", "holding"),
            ("person_drinking_from_cup.png", "drinking from")
        ]
    }
]

# Plot comparisons
for group in comparison_groups:
    fig, axes = plt.subplots(1, len(group["images"]), figsize=(5*len(group["images"]), 5))
    if len(group["images"]) == 1:
        axes = [axes]
    
    fig.suptitle(group["title"], fontsize=16, fontweight='bold')
    
    for ax, (img_name, relationship) in zip(axes, group["images"]):
        img_path = results_dir / img_name
        if img_path.exists():
            img = Image.open(img_path)
            ax.imshow(img)
            ax.set_title(f"Relationship: '{relationship}'", fontsize=12)
            ax.axis('off')
        else:
            ax.text(0.5, 0.5, 'Image not found', ha='center', va='center')
            ax.set_title(img_name)
            ax.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n{group['title']}")
    print("Expected differences:")
    if "Bike" in group["title"]:
        print("  - 'riding': Dynamic action, person actively on bike in motion")
        print("  - 'on': Person sitting on stationary bike")
        print("  - 'next to': Person standing beside bike, spatial separation")
    elif "Hat" in group["title"]:
        print("  - 'wearing': Hat on dog's head, physical contact")
        print("  - 'next to': Hat beside dog, spatial separation")
    elif "Cup" in group["title"]:
        print("  - 'holding': Cup in person's hand, static grasp")
        print("  - 'drinking from': Cup near mouth, drinking action")
    print()

## Analysis Questions

After viewing the generated images, evaluate:

1. **Spatial Arrangement**: Do different relationships result in different object positions?
   - `riding` vs `on` vs `next to` - clear spatial differences?
   
2. **Action vs Static**: Dynamic relationships (`riding`, `drinking`) vs static (`on`, `holding`)?

3. **Contact vs Separation**: Contact relationships (`wearing`, `holding`) vs spatial (`next to`)?

4. **Consistency**: Are the same relationship types consistently rendered across different objects?

5. **Overall Quality**: Do images look realistic and match the prompt?